# 範例 18：評估模型效果

比較原始模型和微調後模型的回答品質！

## 為什麼需要評估？

Fine-Tuning 後需要確認：
- 模型是否真的變好了？
- 哪些方面改善了？
- 有沒有產生新問題？

## 學習目標
- 了解模型評估的方法
- 學會使用 AI 評分
- 建立評估流程

## 前置需求
- LM Studio 運行中，Local Server 已啟動
- 安裝 openai 套件：`pip install openai`

## Step 1: 匯入套件並設定

In [ ]:
from openai import OpenAI
import json

client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"
)

print("已連接到 LM Studio！")

## Step 2: 定義評估用的測試問題

In [ ]:
# 測試問題集
test_questions = [
    "如何在 Python 中讀取 CSV 檔案？",
    "解釋什麼是 API",
    "for 迴圈和 while 迴圈有什麼差別？"
]

print("測試問題：")
for i, q in enumerate(test_questions, 1):
    print(f"{i}. {q}")

## Step 3: 建立回應取得函數

In [ ]:
def get_response(model, question, system_prompt=None):
    """
    取得模型回應
    
    參數：
        model: 模型名稱
        question: 問題
        system_prompt: 系統提示詞（可選）
    """
    messages = []

    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    messages.append({"role": "user", "content": question})

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )

    return response.choices[0].message.content

## Step 4: 建立 AI 評分函數

使用 AI 來評估另一個 AI 的回答品質（這是一種常見的評估方法）

In [ ]:
def evaluate_response(question, response, criteria):
    """
    使用 AI 評估回答品質

    參數：
        question: 原始問題
        response: 模型回答
        criteria: 評估標準

    回傳：
        評估結果（分數和原因）
    """

    prompt = f"""請評估以下回答的品質，給予 1-5 分的評分。

問題：{question}

回答：{response}

評估標準：
{criteria}

評分說明：
- 5 分：優秀，完全符合所有標準
- 4 分：良好，大部分符合標準
- 3 分：普通，基本符合標準
- 2 分：不佳，有明顯問題
- 1 分：很差，完全不符合標準

請用以下 JSON 格式回答：
{{"score": <1-5的數字>, "reason": "評分原因（一句話）"}}

只輸出 JSON："""

    eval_response = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": prompt}]
    )

    try:
        result = eval_response.choices[0].message.content
        # 找出 JSON
        start = result.find('{')
        end = result.rfind('}') + 1
        if start != -1 and end != 0:
            return json.loads(result[start:end])
    except:
        pass
    
    return {"score": 0, "reason": "評估失敗"}

## Step 5: 測試單一問題評估

In [ ]:
# 評估標準
evaluation_criteria = """
- 準確性：回答是否正確
- 完整性：是否涵蓋重要資訊
- 清晰度：是否容易理解
- 實用性：是否提供有用的範例或建議
"""

# 測試
test_question = test_questions[0]
print(f"問題：{test_question}")
print("-" * 50)

# 取得回答
response = get_response("gpt-oss-120b", test_question)
print(f"回答：{response[:200]}...")
print("-" * 50)

# 評估
evaluation = evaluate_response(test_question, response, evaluation_criteria)
print(f"評分：{evaluation.get('score', 'N/A')}/5")
print(f"原因：{evaluation.get('reason', 'N/A')}")

## Step 6: 比較有無系統提示詞的差異

In [ ]:
def compare_with_without_system_prompt(question):
    """
    比較有無系統提示詞的回答差異
    """
    system_prompt = "你是一位專業的程式設計教師，用簡單易懂的方式回答問題，並提供程式碼範例。"
    
    # 無系統提示詞
    response_no_system = get_response("gpt-oss-120b", question)
    eval_no_system = evaluate_response(question, response_no_system, evaluation_criteria)
    
    # 有系統提示詞
    response_with_system = get_response("gpt-oss-120b", question, system_prompt)
    eval_with_system = evaluate_response(question, response_with_system, evaluation_criteria)
    
    return {
        "question": question,
        "no_system": {
            "response": response_no_system,
            "score": eval_no_system.get("score", 0),
            "reason": eval_no_system.get("reason", "")
        },
        "with_system": {
            "response": response_with_system,
            "score": eval_with_system.get("score", 0),
            "reason": eval_with_system.get("reason", "")
        }
    }

In [ ]:
# 比較測試
print("=== 比較有無系統提示詞 ===")
comparison = compare_with_without_system_prompt(test_questions[1])

print(f"\n問題：{comparison['question']}")
print("\n--- 無系統提示詞 ---")
print(f"評分：{comparison['no_system']['score']}/5")
print(f"回答：{comparison['no_system']['response'][:150]}...")

print("\n--- 有系統提示詞 ---")
print(f"評分：{comparison['with_system']['score']}/5")
print(f"回答：{comparison['with_system']['response'][:150]}...")

## Step 7: 批次評估多個問題

In [ ]:
def batch_evaluate(questions, system_prompt=None):
    """
    批次評估多個問題
    """
    results = []
    
    for i, question in enumerate(questions, 1):
        print(f"\n評估第 {i}/{len(questions)} 題...")
        
        # 取得回答
        response = get_response("gpt-oss-120b", question, system_prompt)
        
        # 評估
        evaluation = evaluate_response(question, response, evaluation_criteria)
        
        results.append({
            "question": question,
            "response": response,
            "score": evaluation.get("score", 0),
            "reason": evaluation.get("reason", "")
        })
        
        print(f"  評分：{evaluation.get('score', 'N/A')}/5")
    
    return results

In [ ]:
# 執行批次評估
print("=== 批次評估（有系統提示詞）===")
system_prompt = "你是一位程式設計教師，用簡單易懂的方式回答問題。"

results = batch_evaluate(test_questions, system_prompt)

## Step 8: 統計與報告

In [ ]:
def generate_report(results):
    """
    生成評估報告
    """
    scores = [r["score"] for r in results if r["score"] > 0]
    
    if not scores:
        print("沒有有效的評估結果")
        return
    
    avg_score = sum(scores) / len(scores)
    max_score = max(scores)
    min_score = min(scores)
    
    print("\n" + "=" * 50)
    print("評估報告")
    print("=" * 50)
    print(f"\n評估題數：{len(results)}")
    print(f"平均分數：{avg_score:.2f}/5")
    print(f"最高分數：{max_score}/5")
    print(f"最低分數：{min_score}/5")
    
    # 分數分布
    print("\n分數分布：")
    for score in range(5, 0, -1):
        count = scores.count(score)
        bar = "█" * count
        print(f"  {score} 分：{bar} ({count})")
    
    # 詳細結果
    print("\n詳細結果：")
    for i, r in enumerate(results, 1):
        print(f"\n[{i}] {r['question'][:30]}...")
        print(f"    分數：{r['score']}/5")
        print(f"    評語：{r['reason']}")

In [ ]:
# 生成報告
generate_report(results)

## 評估指標說明

### 常用評估維度

| 維度 | 說明 |
|------|------|
| 準確性 | 資訊是否正確 |
| 完整性 | 是否涵蓋所有重點 |
| 清晰度 | 是否容易理解 |
| 實用性 | 是否有幫助 |
| 相關性 | 是否切題 |

### 評估方法比較

| 方法 | 優點 | 缺點 |
|------|------|------|
| 人工評估 | 最準確 | 耗時、成本高 |
| AI 評估 | 快速、一致 | 可能有偏差 |
| 自動指標 | 客觀、快速 | 不一定反映品質 |

## 重點回顧

1. **評估的重要性**：確認模型是否真正改善

2. **評估方法**：
   - AI 評分（本範例）
   - 人工評估
   - 自動指標

3. **比較方式**：
   - 有無系統提示詞
   - 不同模型
   - Fine-Tuning 前後

4. **注意事項**：
   - 使用多個問題測試
   - 考慮多個評估維度
   - AI 評估可能有偏差

## 下一步

在最後一個範例中，我們將建立一個完整的評估系統！